## **Görev 1: Google Colab'ı Hazırla**

*Adım 1: Gerekli Kütüphaneleri Yükle*

In [ ]:
# Proje için gerekli kütüphaneleri yüklüyoruz
# qwen modeli, lora (peft), ve veri işleme araçları
# Flash Attention'ı çıkardık, T4 GPU ile uyumlu olanları yüklüyoruz
# Bu kodun hatasız çalışması lazım
!pip install torch transformers datasets peft bitsandbytes accelerate

*Adım 2: Hugging Face'e Giriş Yap*

In [ ]:
#Hugging Face'e Giriş Yap
from huggingface_hub import login

# Çalıştırdığında bir kutucuk çıkacak.
# Token'ı oraya yapıştır ve Enter'a bas.
login()

*Adım 3: Base Modeli İndirip Test Et*

In [ ]:
#Modeli İndir ve Test Et (Görev 1)
#Dokümanın istediği Qwen2.5-Coder-1.5B-Instruct modelini indireceğiz ve basit bir soru soracağız.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Dokümanın istediği zorunlu modeli seçiyoruz
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

print("Model indiriliyor... (Bu işlem 1-2 dakika sürebilir)")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16, # Hafıza dostu mod
    device_map="auto" # GPU'yu otomatik kullan
)

# 2. Test sorusu soralım
soru = "Write a python function to check if a number is prime."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": soru}
]

# 3. Cevabı ürettirelim
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=256)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n--- MODELİN CEVABI ---")
print(response)

## **Görev 2: Dataset Analizi**

*Adım 1: Verileri İndir*

In [ ]:
from datasets import load_dataset

print("1. DEEP Dataset indiriliyor...")
# Dokümandaki zorunlu DEEP dataset'i indiriyoruz
dataset_deep = load_dataset("Naholav/CodeGen-Deep-5K", split="train")

print("2. DIVERSE Dataset indiriliyor...")
# Dokümandaki zorunlu DIVERSE dataset'i indiriyoruz
dataset_diverse = load_dataset("Naholav/CodeGen-Diverse-5K", split="train")

print("\n--- İNDİRME TAMAMLANDI ---")
print(f"DEEP Dataset Soru Sayısı: {len(dataset_deep)}")
print(f"DIVERSE Dataset Soru Sayısı: {len(dataset_diverse)}")

# Bir örnek inceleyelim
print("\n--- DEEP DATASET'TEN BİR ÖRNEK ---")
example = dataset_deep[0]
print(f"SORU (Input): {example['input'][:100]}...") # Sorunun başı
print(f"\nÇÖZÜM (Solution): {example['solution'][:100]}...") # Cevabın başı

print("\n--- KRİTİK BİLGİ ---")
print("Dokümana göre eğitimde sadece 'solution' alanını kullanacağız.")

## **Görev 3: Training (Eğitim)**

In [ ]:
#Eğitim kodunu çok daha kolay yazmak için trl adında popüler bir kütüphane kullanacağız.
!pip install -q trl

 ***Adım 1: Eğitimi Başlatıyoruz (DEEP Dataset)***

Aşağıdaki kod bloğu:

-Modeli eğitime hazırlıyor (LoRA ayarlarını yapıyor) .

-Veriyi modelin anlayacağı formata çeviriyor (System Prompt ekliyor).

-Bu kod DEEP Dataset  kullanarak eğitimi başlatacak ve her şeyi /content/drive/MyDrive/qwen-deep-lora-model klasörüne kaydedecek.

In [ ]:
# 1. Google Drive Bağlantısı (Sigorta!)
from google.colab import drive
drive.mount('/content/drive')

# 2. Kütüphaneleri Kur (Sabit Versiyon)
!pip install torch transformers datasets peft bitsandbytes accelerate
!pip install trl==0.8.6

# 3. Hugging Face Girişi
from huggingface_hub import login
login()
# Token'ını gir ve Enter'a bas

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# --- 1. AYARLAR ---
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct" # [cite: 24]
dataset_name = "DEEP_Dataset"

# --- KAYIT YERİ: GOOGLE DRIVE ---
# Bilgisayar kapansa bile silinmez!
output_dir = "/content/drive/MyDrive/qwen-deep-lora-model"

# --- 2. MODEL VE TOKENIZER ---
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# --- 3. LORA AYARLARI ---
# Dokümana uygun rank ve modüller [cite: 91, 94]
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# --- 4. VERİ FORMATLAMA ---
# Zorunlu system prompt [cite: 131]
system_prompt = "You are an expert Python programmer. Please read the problem carefully before writing any Python code."

def formatting_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['input'])):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": examples['input'][i]},
            {"role": "assistant", "content": examples['solution'][i]} # Sadece solution [cite: 44]
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        output_texts.append(text)
    return output_texts

# --- DATASET SEÇİMİ: DEEP ---
print("DEEP Dataset İndiriliyor...")
dataset = load_dataset("Naholav/CodeGen-Deep-5K", split="train") # [cite: 10]

# --- 5. EĞİTİM AYARLARI (Hafıza Dostu & Drive Kayıtlı) ---
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,      # Hafıza için 1 [cite: 181]
    gradient_accumulation_steps=16,     # Dengelemek için 16 [cite: 112]
    gradient_checkpointing=True,        # OOM önleyici [cite: 119]
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# --- 6. BAŞLAT ---
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
    formatting_func=formatting_prompts_func,
    max_seq_length=1024, # Sadece kod için 1024 yeterli [cite: 108]
    packing=False
)

print("--- EĞİTİM BAŞLIYOR (DEEP DATASET) -> DRIVE'A KAYDEDİLİYOR ---")
trainer.train()

# --- 7. SON KAYIT ---
print("Eğitim bitti, son kontroller yapılıyor...")
trainer.model.save_pretrained(output_dir)
print(f"TEBRİKLER! Modeliniz şurada güvenle saklandı: {output_dir}")

 ***Adım 2: Eğitimi Başlatıyoruz (DIVERSE Dataset)***

Aşağıdaki kod bloğu:

-Bu kod DIVERSE Dataset  kullanarak eğitimi başlatacak ve her şeyi /content/drive/MyDrive/qwen-deep-lora-model klasörüne kaydedecek.

In [ ]:
# --- 1. KURULUMLAR VE BAĞLANTILAR ---
print("Ortam hazırlanıyor...")
!pip install -q torch transformers datasets peft bitsandbytes accelerate
!pip install -q trl==0.8.6

from google.colab import drive
drive.mount('/content/drive')

from huggingface_hub import login
login() # Token'ını girmen gerekecek


In [ ]:
# --- 2. EĞİTİM KODU (DIVERSE DATASET) ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# Ayarlar
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
dataset_name = "DIVERSE_Dataset" # [cite: 12]
output_dir = "/content/drive/MyDrive/qwen-diverse-lora-model" # Drive'a yeni klasör

# Model ve Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# LoRA Ayarları
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# Veri Formatlama
system_prompt = "You are an expert Python programmer. Please read the problem carefully before writing any Python code."

def formatting_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['input'])):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": examples['input'][i]},
            {"role": "assistant", "content": examples['solution'][i]}
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        output_texts.append(text)
    return output_texts

# DIVERSE Dataset İndiriliyor [cite: 12]
print("DIVERSE Dataset İndiriliyor...")
dataset = load_dataset("Naholav/CodeGen-Diverse-5K", split="train")

# Eğitim Ayarları (Hafıza Dostu)
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# Başlat
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
    formatting_func=formatting_prompts_func,
    max_seq_length=1024,
    packing=False
)

print("--- EĞİTİM BAŞLIYOR (DIVERSE) -> DRIVE'A KAYDEDİLİYOR ---")
trainer.train()

print("Eğitim bitti, kaydediliyor...")
trainer.model.save_pretrained(output_dir)
print(f"HARİKA! DIVERSE modeli de şuraya kaydedildi: {output_dir}")

In [ ]:
from huggingface_hub import login, HfApi

# 1. Önce Giriş Yapalım (Emin olmak için)
print("Hugging Face'e giriş yapılıyor...")
login()

# --- AYARLAR (BURAYI DÜZENLE!) ---
# Lütfen tırnak içine kendi Hugging Face kullanıcı adını yaz
username = "caglademir"
# Örnek: username = "ahmetyilmaz"

api = HfApi()

# Yükleme Fonksiyonu
def modele_yukle(drive_path, model_name):
    repo_id = f"{username}/{model_name}"
    print(f"\n🚀 Başlatılıyor: {repo_id}")
    print(f"Kaynak: {drive_path}")

    try:
        # Repoyu oluştur (varsa hata verme)
        api.create_repo(repo_id=repo_id, exist_ok=True, repo_type="model")

        # Dosyaları yükle
        api.upload_folder(
            folder_path=drive_path,
            repo_id=repo_id,
            repo_type="model"
        )
        print(f"✅ BAŞARILI! Modelin şurada yayında:")
        print(f"👉 https://huggingface.co/{repo_id}")
    except Exception as e:
        print(f"❌ Hata oluştu: {e}")

# --- İŞLEM BAŞLIYOR ---

# 1. DEEP Modeli Yükle
modele_yukle(
    drive_path="/content/drive/MyDrive/qwen-deep-lora-model",
    model_name="Qwen2.5-Coder-1.5B-LoRA-DEEP"
)

# 2. DIVERSE Modeli Yükle
modele_yukle(
    drive_path="/content/drive/MyDrive/qwen-diverse-lora-model",
    model_name="Qwen2.5-Coder-1.5B-LoRA-DIVERSE"
)